In [1]:
##### reproducibility for Keras
import os
import numpy as np
import random as rn
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1,
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)

from keras import backend as K

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

##### Import some librarys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
df = pd.read_csv('../data/feature_selection_positive.csv')


# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [7]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs.h5')

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 2s 1ms/step - loss: 1.8700 - acc: 0.2507 - val_loss: 1.4152 - val_acc: 0.3155
Epoch 2/100
1340/1340 [==============================] - 0s 74us/step - loss: 1.7653 - acc: 0.2582 - val_loss: 1.3522 - val_acc: 0.2976
Epoch 3/100
1340/1340 [==============================] - 0s 67us/step - loss: 1.6996 - acc: 0.2813 - val_loss: 1.3382 - val_acc: 0.2887
Epoch 4/100
1340/1340 [==============================] - 0s 69us/step - loss: 1.6141 - acc: 0.2985 - val_loss: 1.3362 - val_acc: 0.2976
Epoch 5/100
1340/1340 [==============================] - 0s 74us/step - loss: 1.5480 - acc: 0.3075 - val_loss: 1.3222 - val_acc: 0.3065
Epoch 6/100
1340/1340 [==============================] - 0s 69us/step - loss: 1.5700 - acc: 0.3299 - val_loss: 1.3119 - val_acc: 0.3214
Epoch 7/100
1340/1340 [==============================] - 0s 67us/step - loss: 1.5446 - acc: 0.3209 - val_loss: 1.2941 - val_acc: 0.4286
Ep

In [10]:
#read data
df = pd.read_csv('../data/feature_selection_negative.csv')

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [15]:
tf.set_random_seed(42)

# make keras model

inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs.h5')

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 4ms/step - loss: 2.1786 - acc: 0.2225 - val_loss: 1.3061 - val_acc: 0.4149
Epoch 2/100
373/373 [==============================] - 0s 61us/step - loss: 1.9127 - acc: 0.2654 - val_loss: 1.3591 - val_acc: 0.4574
Epoch 3/100
373/373 [==============================] - 0s 89us/step - loss: 1.8433 - acc: 0.3137 - val_loss: 1.5017 - val_acc: 0.4894
Epoch 4/100
373/373 [==============================] - 0s 71us/step - loss: 1.6841 - acc: 0.3592 - val_loss: 1.6397 - val_acc: 0.5106
Epoch 5/100
373/373 [==============================] - 0s 74us/step - loss: 1.6561 - acc: 0.3753 - val_loss: 1.6861 - val_acc: 0.5000
Epoch 6/100
373/373 [==============================] - 0s 71us/step - loss: 1.7015 - acc: 0.3700 - val_loss: 1.6637 - val_acc: 0.5000
Epoch 7/100
373/373 [==============================] - 0s 53us/step - loss: 1.6823 - acc: 0.3753 - val_loss: 1.5997 - val_acc: 0.5000
Epoch 8/100
373/37